In [134]:
import numpy as np
import heapq
import random
from collections import defaultdict, Counter
import requests
import random

# Function to download text data from a URL and save to a file
def download_text(url, filename):
    response = requests.get(url, allow_redirects=True)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        return True
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
        return False

# Function to read text data from file
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Function to calculate character frequencies
def calculate_frequencies(text):
    return Counter(text)

def generate_huffman_codes(freq_dict):
  heap = [[wt,[sym,""]] for sym,wt in freq_dict.items()]
  heapq.heapify(heap)

  while(len(heap) > 1):
    lo=heapq.heappop(heap)
    for pair in lo[1:]:
      pair[1]='0' + pair[1] # if '' then '0', if '0' then '00' if '1' then '01' etc.
    hi=heapq.heappop(heap)
    for pair in hi[1:]:
      pair[1]='1' +pair[1] # if '' then '0', if '0' then '00' if '1' then '01' etc.
    heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])

  huffman_codes = {}
  for sym, code in heap[0][1:]:
    huffman_codes[sym] = code # assigning keys to symbols in freq dict and their values as their huffman code

  return huffman_codes

def build_huffman_tree(codes): # binary tree implementation
  huffman_tree = {}

  for sym, code in codes.items(): # sym=key, code=value
    tree_node = huffman_tree # root initialisation
    for bit in code:
      if bit not in tree_node:
        tree_node[bit] = {} # new dictionary for each node that has not been added to tree
      tree_node = tree_node[bit] # move to next reference
    tree_node['symbol'] = sym # node represents given symbol/character

  return huffman_tree

def decode_huffman(encoded_text, tree):
  decoded_text = ""
  tree_node = tree # root initialisation
  for bit in encoded_text:
    if bit in tree_node:
      tree_node = tree_node[bit] # moves down tree
      if 'symbol' in tree_node: # checking if it is at the leaf level
        decoded_text += tree_node['symbol'] # adding symbol to text
        tree_node = tree # returning to root

  return decoded_text


def simulate_channel_transmission(code, error_probability=0.0):
  transmitted_code = ""
  for sym in code:
    if random.random() < error_probability: # checking if the random probabiblity (0-1) is less than the error
      if sym == '0':
        transmitted_code += '1'  # flips bit if error goes through
      else:
        transmitted_code += '0'
    else:
      transmitted_code += sym # stays the same if no error
  return transmitted_code


def generate_shannon_codes(freq_dict):
  sorted_freq = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True) # sorting frequencies in descending order

  def decTobinary(m,decimalPoint=False): # internal helper method to turn decimal to binary
    intpart = False # if x>=1 then this will take the value of the integer part
    signStr = "" #if input is negative then use this as first character in output
    if decimalPoint:
        decimalPointStr="0."
    else:
        decimalPointStr=""

    if (m<0):
        m=-m
        NEGATIVE=True
        signStr = "-"

    if (m>1):
        intpart = int(m)
        binIntPart = np.binary_repr(intpart)
        m-=intpart
    #assert (m<1), "input must be a positive number less than 1 i.e. 0<x<1"
    if m==0:
        return "000000000000000000000000000000000000"
    binary = []
    n = m-int(m)

    while n != 0:
        x = 2 * n #shift to left
        y = int(x) #be 0 or 1 remember 10^0 and 2^0 = 1
        binary.append(y)
        n = x-y #x is shifted left value, y is integer part
    if intpart:
        return signStr+binIntPart+"."+"".join([str(i) for i in binary]) #I want the output to be a string
    else:
        return signStr+decimalPointStr+"".join([str(i) for i in binary]) #I want the output to be a string

  def ShannonFanoCk(k,probs,PRINT=False,OUTPUT_TABLE=False): # internal helper method to find codes from
    #assert sum(probs)<=1, "probabilities should add to less than 1."
    codeLen = int(np.ceil(np.log2(1/probs[k])))+1    #determine the length of the code: Note the +1 for Shannon Fano Elias
    Fx = sum(probs[0:k])
    Fxbar = np.round(Fx +probs[k]/2, 14)
    bin1=decTobinary(Fxbar)+"000000000000" #sometimes binary numbers are short, e.g. 0.5  = 0.1 in binary => add extra zeros 0.10000
    if PRINT:
        print(codeLen,", Fbar(x)=",Fxbar,", FbarBin(x)=",bin1[0:codeLen])
    if OUTPUT_TABLE:
      return [k+1, probs[k],Fx,Fxbar,codeLen,bin1[0:codeLen]]
    else:
      return bin1[0:codeLen]

  total_freq = sum(freq_dict.values())
  data = [freq / total_freq for _, freq in sorted_freq] # finding probs
  code_dict = {sorted_freq[i][0]: ShannonFanoCk(i, data, False, False) for i in range(len(data))} # generating codes for each symbol

  return code_dict


# Main execution flow (You may not need to modify this part)
url = 'https://www.gutenberg.org/cache/epub/6150/pg6150.txt'
filename = 'pg6150.txt'

if download_text(url, filename):
    #TESTING
    #text1 = 'abbdcad'
    #freq_dict1 = calculate_frequencies(text1)
    #print(freq_dict1)
    #huffman_codes1 = generate_huffman_codes(freq_dict1)
    #print(huffman_codes1)
    #huffman_tree1=build_huffman_tree(huffman_codes1)
    #print(huffman_tree1)
    #encoded_text_huffman1 = ''.join(huffman_codes1[sym] for sym in text1)
    #print(encoded_text_huffman1)
    #transmitted_code_huffman1 = simulate_channel_transmission(encoded_text_huffman1)
    #print(transmitted_code_huffman1)
    #decoded_text_huffman1 = decode_huffman(transmitted_code_huffman1, huffman_tree1)
    #print(decoded_text_huffman1)

    #shannon_codes1 = generate_shannon_codes(freq_dict1)
    #print(shannon_codes1)

    #freq_dict2 = {'a': 100, 'b': 60, 'c': 10, 'd': 5, 'e': 25}
    #codes2 = generate_shannon_codes(freq_dict2)
    #print(codes2)

    text_data = read_text_file(filename)
    freq_dict = calculate_frequencies(text_data)

    # Generate Huffman codes and build tree
    huffman_codes = generate_huffman_codes(freq_dict)
    huffman_tree = build_huffman_tree(huffman_codes)

    # Generate Shannon codes
    shannon_codes = generate_shannon_codes(freq_dict)

    # Encode text using Huffman codes
    encoded_text_huffman = ''.join(huffman_codes[char] for char in text_data)

    # Simulate transmitting encoded data and decode it
    transmitted_code_huffman = simulate_channel_transmission(encoded_text_huffman)
    decoded_text_huffman = decode_huffman(transmitted_code_huffman, huffman_tree)

    print("Original Text Sample:", text_data[:500])
    print("Decoded Huffman Text Sample:", decoded_text_huffman[:500])
else:
    print("Text download failed.")

# Use the following code to produce the table of code.

import pandas as pd
import numpy as np
from collections import Counter
import math

# Assume freq_dict, huffman_codes, and shannon_codes are already generated as per previous steps

def calculate_entropy(probabilities):
    return {char: -prob * math.log2(prob) if prob > 0 else 0 for char, prob in probabilities.items()}

def create_table(freq_dict, huffman_codes, shannon_codes):
    total = sum(freq_dict.values())
    probabilities = {char: freq / total for char, freq in freq_dict.items()}
    entropies = calculate_entropy(probabilities)

    # Creating DataFrame
    data = {
        'Character': list(freq_dict.keys()),
        'Frequency': list(freq_dict.values()),
        'Probability': [probabilities[char] for char in freq_dict.keys()],
        'Huffman Code': [huffman_codes[char] for char in freq_dict.keys()],
        'Shannon Code': [shannon_codes[char] for char in freq_dict.keys()],
        'Entropy h_i': [entropies[char] for char in freq_dict.keys()]
    }
    df = pd.DataFrame(data)
    return df

# Example call
df = create_table(freq_dict, huffman_codes, shannon_codes)
df

Original Text Sample: ﻿The Project Gutenberg eBook of The Iliad
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Th
Decoded Huffman Text Sample: ﻿The Project Gutenberg eBook of The Iliad
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where

,Character,Frequency,Probability,Huffman Code,Shannon Code,Entropy h_i
0,﻿,1,0.000001,1011010000100110110,111111111111111111001,0.000023
1,T,5652,0.006690,1000001,111011110,0.048325
2,h,46551,0.055097,0110,011000,0.230409
3,e,76144,0.090123,1111,00110,0.312903
4,,129255,0.152984,110,0001,0.414365
...,...,...,...,...,...,...
87,9,8,0.000009,11100001100010000,111111111111111001,0.000158
88,™,57,0.000067,10110100001011,111111111110001,0.000935
89,•,4,0.000005,10110100001001100,1111111111111111011,0.000084
90,%,1,0.000001,10110100001001101111,111111111111111111110,0.000023


In [ ]:
my_map = {'a': 1, 'b': 2}
inv_map = {v: k for k, v in my_map.items()}
inv_map

{1: 'a', 2: 'b'}

Original Text Sample: ﻿The Project Gutenberg eBook of The Iliad
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Th
Decoded Huffman Text Sample: ﻿The Project Gnaew r, n eBook of The Iliad
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg Lic NedahcHded with gdsooenook or online
at www.gutenberg.org. If you ayepercr ltreo in the United States,
you will have to cheiou the laws of the country where 

,Character,Frequency,Probability,Huffman Code,Shannon Code,Entropy h_i
0,﻿,1,0.000001,1011010000100110110,11111111111111111101,0.000023
1,T,5652,0.006690,1000001,11110000,0.048325
2,h,46551,0.055097,0110,01101,0.230409
3,e,76144,0.090123,1111,0011,0.312903
4,,129255,0.152984,110,001,0.414365
...,...,...,...,...,...,...
87,9,8,0.000009,11100001100010000,11111111111111101,0.000158
88,™,57,0.000067,10110100001011,11111111111001,0.000935
89,•,4,0.000005,10110100001001100,111111111111111110,0.000084
90,%,1,0.000001,10110100001001101111,100000000000000000000,0.000023
